# Convolutional Neural Networks

## Project: Write an Algorithm for Landmark Classification

---

In this notebook, some template code has already been provided for you, and you will need to implement additional functionality to successfully complete this project. You will not need to modify the included code beyond what is requested. Sections that begin with **'(IMPLEMENTATION)'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section, and the specifics of the implementation are marked in the code block with a 'TODO' statement. Please be sure to read the instructions carefully! 

> **Note**: Once you have completed all the code implementations, you need to finalize your work by exporting the Jupyter Notebook as an HTML document. Before exporting the notebook to HTML, all the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to **File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut.  Markdown cells can be edited by double-clicking the cell to enter edit mode.

The rubric contains _optional_ "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. If you decide to pursue the "Stand Out Suggestions", you should include the code in this Jupyter notebook.

---
### Why We're Here

Photo sharing and photo storage services like to have location data for each photo that is uploaded. With the location data, these services can build advanced features, such as automatic suggestion of relevant tags or automatic photo organization, which help provide a compelling user experience. Although a photo's location can often be obtained by looking at the photo's metadata, many photos uploaded to these services will not have location metadata available. This can happen when, for example, the camera capturing the picture does not have GPS or if a photo's metadata is scrubbed due to privacy concerns.

If no location metadata for an image is available, one way to infer the location is to detect and classify a discernible landmark in the image. Given the large number of landmarks across the world and the immense volume of images that are uploaded to photo sharing services, using human judgement to classify these landmarks would not be feasible.

In this notebook, you will take the first steps towards addressing this problem by building models to automatically predict the location of the image based on any landmarks depicted in the image. At the end of this project, your code will accept any user-supplied image as input and suggest the top k most relevant landmarks from 50 possible landmarks from across the world. The image below displays a potential sample output of your finished project.

![Sample landmark classification output](images/sample_landmark_output.png)


### The Road Ahead

We break the notebook into separate steps.  Feel free to use the links below to navigate the notebook.

* [Step 0](#step0): Download Datasets and Install Python Modules
* [Step 1](#step1): Create a CNN to Classify Landmarks (from Scratch)
* [Step 2](#step2): Create a CNN to Classify Landmarks (using Transfer Learning)
* [Step 3](#step3): Write Your Landmark Prediction Algorithm

---
<a id='step0'></a>
## Step 0: Download Datasets and Install Python Modules

**Note: if you are using the Udacity workspace, *YOU CAN SKIP THIS STEP*. The dataset can be found in the `/data` folder and all required Python modules have been installed in the workspace.**

Download the [landmark dataset](https://udacity-dlnfd.s3-us-west-1.amazonaws.com/datasets/landmark_images.zip).
Unzip the folder and place it in this project's home directory, at the location `/landmark_images`.

Install the following Python modules:
* cv2
* matplotlib
* numpy
* PIL
* torch
* torchvision

---

<a id='step1'></a>
## Step 1: Create a CNN to Classify Landmarks (from Scratch)

In this step, you will create a CNN that classifies landmarks.  You must create your CNN _from scratch_ (so, you can't use transfer learning _yet_!), and you must attain a test accuracy of at least 20%.

Although 20% may seem low at first glance, it seems more reasonable after realizing how difficult of a problem this is. Many times, an image that is taken at a landmark captures a fairly mundane image of an animal or plant, like in the following picture.

<img src="images/train/00.Haleakala_National_Park/084c2aa50d0a9249.jpg" alt="Bird in Haleakalā National Park" style="width: 400px;"/>

Just by looking at that image alone, would you have been able to guess that it was taken at the Haleakalā National Park in Hawaii?

An accuracy of 20% is significantly better than random guessing, which would provide an accuracy of just 2%. In Step 2 of this notebook, you will have the opportunity to greatly improve accuracy by using transfer learning to create a CNN.

Remember that practice is far ahead of theory in deep learning.  Experiment with many different architectures, and trust your intuition.  And, of course, have fun!

### (IMPLEMENTATION) Specify Data Loaders for the Landmark Dataset

Use the code cell below to create three separate [data loaders](http://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader): one for training data, one for validation data, and one for test data. Randomly split the images located at `landmark_images/train` to create the train and validation data loaders, and use the images located at `landmark_images/test` to create the test data loader.

All three of your data loaders should be accessible via a dictionary named `loaders_scratch`. Your train data loader should be at `loaders_scratch['train']`, your validation data loader should be at `loaders_scratch['valid']`, and your test data loader should be at `loaders_scratch['test']`.

You may find [this documentation on custom datasets](https://pytorch.org/docs/stable/torchvision/datasets.html#datasetfolder) to be a useful resource.  If you are interested in augmenting your training and/or validation data, check out the wide variety of [transforms](http://pytorch.org/docs/stable/torchvision/transforms.html?highlight=transform)!

In [1]:
import os
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [5]:
### TODO: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes
data_dir = 'directory to extract/landmark_images/'
train_dir = os.path.join(data_dir, 'train/')
test_dir = os.path.join(data_dir, 'test/')


train_transform = transforms.Compose([transforms.Resize(224),
                                transforms.CenterCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
valid_test_transform = transforms.Compose([transforms.Resize(224),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


###### define train and test data
train_data = datasets.ImageFolder(train_dir, transform=train_transform)
valid_data = datasets.ImageFolder(train_dir, transform=valid_test_transform) #####valid data and test data cannot use the same transformation as train data
test_data = datasets.ImageFolder(test_dir, transform=valid_test_transform)


###### split train data to train and validation
valid_size = 0.2 #####define validation sample size %
num_train = len(train_data) ##### training data size
print (num_train)
indices = list(range(num_train)) ####### create indices on training dataset
np.random.shuffle(indices) ###### randomize indices
split = int(np.floor(valid_size * num_train)) ###### number of observations * % of val = num validation
print(split)
train_idx, valid_idx = indices[split:], indices[:split] ###### records after split is train; records before split is validation
train_sampler = SubsetRandomSampler(train_idx) 
valid_sampler = SubsetRandomSampler(valid_idx) ##### this is from torch.utils.data.sampler
                                              ##### Samples elements randomly from a given list of indices, without replacement.
print(type(train_sampler))
    
batch_size=20
num_workers=0

########### create Python iterable over the datasets
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,  num_workers=num_workers)
print(type(train_loader))
loaders_scratch = {'train': train_loader, 'valid': valid_loader, 'test': test_loader}
loaders_scratch
print(type(train_data))
print(type(loaders_scratch))

4996
999
<class 'torch.utils.data.sampler.SubsetRandomSampler'>
<class 'torch.utils.data.dataloader.DataLoader'>
<class 'torchvision.datasets.folder.ImageFolder'>
<class 'dict'>


**Question 1:** Describe your chosen procedure for preprocessing the data. 
- How does your code resize the images (by cropping, stretching, etc)?  What size did you pick for the input tensor, and why?
- Did you decide to augment the dataset?  If so, how (through translations, flips, rotations, etc)?  If not, why not?

**Answer**: 

### (IMPLEMENTATION) Visualize a Batch of Training Data

Use the code cell below to retrieve a batch of images from your train data loader, display at least 5 images simultaneously, and label each displayed image with its class name (e.g., "Golden Gate Bridge").

Visualizing the output of your data loader is a great way to ensure that your data loading and preprocessing are working as expected.

In [11]:
import matplotlib.pyplot as plt
%matplotlib inline

## TODO: visualize a batch of the train data loader

## the class names can be accessed at the `classes` attribute
## of your dataset object (e.g., `train_dataset.classes`)

# classes are folders in each directory with these names, when load images, the class names are taken from folder names
classes = train_data.classes

# obtain one batch of training images
dataiter = iter(train_loader)

images, labels = dataiter.next()
images = images.numpy()
print(images.shape) ############# images with 3x224x224 size

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(0.5, 0.5))
for idx in np.arange(5):
    ax = fig.add_subplot(2, 10, idx+1, xticks=[], yticks=[])
    plt.imshow(np.transpose(images[idx], (1, 2, 0)))
    ax.set_title(classes[labels[idx]])

(20, 3, 224, 224)
Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f24193c27b8> (for post_execute):


ValueError: Floating point image RGB values must be in the 0..1 range.

ValueError: Floating point image RGB values must be in the 0..1 range.

### Initialize use_cuda variable

In [12]:
# useful variable that tells us whether we should use the GPU
use_cuda = torch.cuda.is_available()

### (IMPLEMENTATION) Specify Loss Function and Optimizer

Use the next code cell to specify a [loss function](http://pytorch.org/docs/stable/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/stable/optim.html).  Save the chosen loss function as `criterion_scratch`, and fill in the function `get_optimizer_scratch` below.

In [13]:
## TODO: select loss function
import torch.optim as optim
import torch.nn as nn

# specify loss function (categorical cross-entropy)
criterion_scratch = nn.CrossEntropyLoss()

def get_optimizer_scratch(model):
    ## TODO: select and return an optimizer  
    return optim.SGD(model.parameters(), lr=0.0001)


### (IMPLEMENTATION) Model Architecture

Create a CNN to classify images of landmarks.  Use the template in the code cell below.

In [14]:
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):
    ## TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
        
        ## Define layers of a CNN
        
        # convolutional layer (sees 224x224x3 image tensor)
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1) ####### input layer depth=3; to produce 16 filters; use filters 3x3 in size
                                                    ####### image size does not change after conv layer
                                                    ####### padding= (filter_size-1)/2
        # convolutional layer (sees 16x16x16 tensor)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        # convolutional layer (sees 8x8x32 tensor)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)   ###### Max polling layer use a 2x2 filter (kernal size); 
                                         ###### step by 2; so reduce size of image by 2 (stride size)
        # linear layer (64 * 28 * 28 -> 500)
        self.fc1 = nn.Linear(64 * 28 * 28, 1000)  ######## define fully connected layer 1
        # linear layer (500 -> 50)
        self.fc2 = nn.Linear(1000, 500) ######## define fully connected layer 2; output is 50 categories
        # dropout layer (p=0.2)
        self.fc3 = nn.Linear(500, 50) ######## define fully connected layer 2; output is 50 categories
        # dropout layer (p=0.2)
        self.dropout = nn.Dropout(0.2)
        
        
    
    def forward(self, x):
        ## Define forward behavior
        x=self.pool(F.relu(self.conv1(x))) ##### 1st convolutional+pooling layer
        x=self.pool(F.relu(self.conv2(x))) ##### 2nd convolutional+pooling layer
        x=self.pool(F.relu(self.conv3(x))) ##### 3rd convolutional+pooling layer
        x=x.view(-1,64*28*28) ##### use -1 to let Python calculate the inferred shape
        x = self.dropout(x) ##### add dropout layer
        x = F.relu(self.fc1(x)) ##### first hidden layer fully connected
        x = self.dropout(x) ##### add dropout laywer        
        x = F.relu(self.fc2(x))    ##### add 2nd hidden layer, with relu activation function   
        x = self.dropout(x) ##### add dropout laywer        
        x = self.fc3(x)   ##### add 2nd hidden layer, with relu activation function   
        
        return x

#-#-# Do NOT modify the code below this line. #-#-#

# instantiate the CNN
model_scratch = Net()
print (model_scratch)

# move tensors to GPU if CUDA is available
if use_cuda:
    model_scratch.cuda()

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=500, bias=True)
  (fc3): Linear(in_features=500, out_features=50, bias=True)
  (dropout): Dropout(p=0.2)
)


__Question 2:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  

__Answer:__
1. Define convolutional laywers
2. Define Pooling layer
3. Define dropout layers
4. Define linear layers
4. Construct the CNN: I used 3 convolutional layers and after each, a pooling layer is applied; after the last step; I resized the output into an array; after that, apply 2 fully connected layers with drop out 

In [15]:
loaders_scratch['train']

### (IMPLEMENTATION) Implement the Training Algorithm

Implement your training algorithm in the code cell below.  [Save the final model parameters](http://pytorch.org/docs/master/notes/serialization.html) at the filepath stored in the variable `save_path`.

In [16]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        print ("---n_epochs---")
        print(epoch)
        
        ###################
        # train the model #
        ###################
        # set the module to training mode
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']): ##### batch_idx is the index, (data,target) is the value
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            
            ##print (batch_idx)

            ## TODO: find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - train_loss))
            
            ####### clear the gradients of all optimized variables
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            # update training loss
            ###### train_loss += loss.item()*data.size(0)
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - train_loss))

            
            

        ######################    
        # validate the model #
        ######################
        # set the model to evaluation mode
        
        ######## Enumerate() method adds a counter to an iterable and returns it in
        ######## a form of enumerating object. 
        ######## This enumerated object can then be used directly for loops or converted 
        ######## into a list of tuples using the list() method.
        
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            ## TODO: update average validation loss 
            output = model(data) ###### for each image in validation set, calculate the prediction
            loss = criterion(output, target) ##### calculate the loss between output and target
            # update average validation loss 
            ## valid_loss += loss.item()*data.size(0)
            valid_loss += ((1 / (batch_idx + 1)) * (loss.data.item() - valid_loss))    
    
        # calculate average losses
        ##train_loss = train_loss/len(train_loader.sampler)
        ##valid_loss = valid_loss/len(valid_loader.sampler)            

        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
                epoch, 
                train_loss,
                valid_loss
             ))

        ## TODO: if the validation loss has decreased, save the model at the filepath stored in save_path
        
        # save model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), 'model_scratch.pt')
            valid_loss_min = valid_loss       
        
    return model

### (IMPLEMENTATION) Experiment with the Weight Initialization

Use the code cell below to define a custom weight initialization, and then train with your weight initialization for a few epochs. Make sure that neither the training loss nor validation loss is `nan`.

Later on, you will be able to see how this compares to training with PyTorch's default weight initialization.

In [17]:
def custom_weight_init(m):
    ## TODO: implement a weight initialization strategy
    classname = m.__class__.__name__
    # for every Linear layer in a model..
    if classname.find('Linear') != -1:
        # get the number of the inputs
        n = m.in_features
        y = (1.0/np.sqrt(n))
        m.weight.data.normal_(0, y)
        m.bias.data.fill_(0)    
    

#-#-# Do NOT modify the code below this line. #-#-#
    
model_scratch.apply(custom_weight_init)
model_scratch = train(20, loaders_scratch, model_scratch, get_optimizer_scratch(model_scratch),
                      criterion_scratch, use_cuda, 'ignore.pt')

---n_epochs---
1
Epoch: 1 	Training Loss: 3.913270 	Validation Loss: 3.912607
Validation loss decreased (inf --> 3.912607).  Saving model ...
---n_epochs---
2
Epoch: 2 	Training Loss: 3.913000 	Validation Loss: 3.912379
Validation loss decreased (3.912607 --> 3.912379).  Saving model ...
---n_epochs---
3
Epoch: 3 	Training Loss: 3.911887 	Validation Loss: 3.912160
Validation loss decreased (3.912379 --> 3.912160).  Saving model ...
---n_epochs---
4
Epoch: 4 	Training Loss: 3.911702 	Validation Loss: 3.911948
Validation loss decreased (3.912160 --> 3.911948).  Saving model ...
---n_epochs---
5
Epoch: 5 	Training Loss: 3.911292 	Validation Loss: 3.911738
Validation loss decreased (3.911948 --> 3.911738).  Saving model ...
---n_epochs---
6
Epoch: 6 	Training Loss: 3.912560 	Validation Loss: 3.911462
Validation loss decreased (3.911738 --> 3.911462).  Saving model ...
---n_epochs---
7
Epoch: 7 	Training Loss: 3.911753 	Validation Loss: 3.911251
Validation loss decreased (3.911462 --> 3.911

### (IMPLEMENTATION) Train and Validate the Model

Run the next code cell to train your model.

In [18]:
## TODO: you may change the number of epochs if you'd like,
## but changing it is not required
num_epochs = 100

#-#-# Do NOT modify the code below this line. #-#-#

# function to re-initialize a model with pytorch's default weight initialization
def default_weight_init(m):
    reset_parameters = getattr(m, 'reset_parameters', None)
    if callable(reset_parameters):
        m.reset_parameters()

# reset the model parameters
model_scratch.apply(default_weight_init)

# train the model
model_scratch = train(num_epochs, loaders_scratch, model_scratch, get_optimizer_scratch(model_scratch), 
                      criterion_scratch, use_cuda, 'model_scratch.pt')

---n_epochs---
1
Epoch: 1 	Training Loss: 3.912212 	Validation Loss: 3.912353
Validation loss decreased (inf --> 3.912353).  Saving model ...
---n_epochs---
2
Epoch: 2 	Training Loss: 3.912213 	Validation Loss: 3.912327
Validation loss decreased (3.912353 --> 3.912327).  Saving model ...
---n_epochs---
3
Epoch: 3 	Training Loss: 3.912009 	Validation Loss: 3.912316
Validation loss decreased (3.912327 --> 3.912316).  Saving model ...
---n_epochs---
4
Epoch: 4 	Training Loss: 3.912212 	Validation Loss: 3.912285
Validation loss decreased (3.912316 --> 3.912285).  Saving model ...
---n_epochs---
5
Epoch: 5 	Training Loss: 3.911933 	Validation Loss: 3.912269
Validation loss decreased (3.912285 --> 3.912269).  Saving model ...
---n_epochs---
6
Epoch: 6 	Training Loss: 3.911936 	Validation Loss: 3.912248
Validation loss decreased (3.912269 --> 3.912248).  Saving model ...
---n_epochs---
7
Epoch: 7 	Training Loss: 3.911877 	Validation Loss: 3.912232
Validation loss decreased (3.912248 --> 3.912

### (IMPLEMENTATION) Test the Model

Run the code cell below to try out your model on the test dataset of landmark images. Run the code cell below to calculate and print the test loss and accuracy.  Ensure that your test accuracy is greater than 20%.

In [19]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    # set the module to evaluation mode
    model.eval()

    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# load the model that got the best validation accuracy
model_scratch.load_state_dict(torch.load('model_scratch.pt'))
test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)

Test Loss: 3.907136


Test Accuracy:  3% (42/1250)


---
<a id='step2'></a>
## Step 2: Create a CNN to Classify Landmarks (using Transfer Learning)

You will now use transfer learning to create a CNN that can identify landmarks from images.  Your CNN must attain at least 60% accuracy on the test set.

### (IMPLEMENTATION) Specify Data Loaders for the Landmark Dataset

Use the code cell below to create three separate [data loaders](http://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader): one for training data, one for validation data, and one for test data. Randomly split the images located at `landmark_images/train` to create the train and validation data loaders, and use the images located at `landmark_images/test` to create the test data loader.

All three of your data loaders should be accessible via a dictionary named `loaders_transfer`. Your train data loader should be at `loaders_transfer['train']`, your validation data loader should be at `loaders_transfer['valid']`, and your test data loader should be at `loaders_transfer['test']`.

If you like, **you are welcome to use the same data loaders from the previous step**, when you created a CNN from scratch.

In [15]:
m=model_scratch.load_state_dict(torch.load('model_scratch.pt'))

In [16]:
type(m)

NoneType

In [17]:
import torchvision
from torchvision import datasets, models, transforms

In [18]:
### TODO: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes



###### split train data to train and validation
valid_size = 0.3 #####define validation sample size %
num_train = len(train_data) ##### training data size
print (num_train)
indices = list(range(num_train)) ####### create indices on training dataset
np.random.shuffle(indices) ###### randomize indices
split = int(np.floor(valid_size * num_train)) ###### number of observations * % of val = num validation
print(split)
train_idx, valid_idx = indices[split:], indices[:split] ###### records after split is train; records before split is validation
train_sampler = SubsetRandomSampler(train_idx) 
valid_sampler = SubsetRandomSampler(valid_idx) ##### this is from torch.utils.data.sampler
                                              ##### Samples elements randomly from a given list of indices, without replacement.
print(type(train_sampler))
    
batch_size=20
num_workers=0

########### create Python iterable over the datasets
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)
print(type(train_loader))
loaders_transfer = {'train': train_loader, 'valid': valid_loader, 'test': test_loader}
print(loaders_transfer )



4996
1498
<class 'torch.utils.data.sampler.SubsetRandomSampler'>
<class 'torch.utils.data.dataloader.DataLoader'>
{'train': <torch.utils.data.dataloader.DataLoader object at 0x7f6450638710>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x7f64506385c0>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7f6450638748>}


### (IMPLEMENTATION) Specify Loss Function and Optimizer

Use the next code cell to specify a [loss function](http://pytorch.org/docs/stable/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/stable/optim.html).  Save the chosen loss function as `criterion_transfer`, and fill in the function `get_optimizer_transfer` below.

In [19]:
## TODO: select loss function
criterion_transfer = nn.CrossEntropyLoss()
def get_optimizer_transfer(model):
    ## TODO: select and return optimizer
    return optim.SGD(model.parameters(), lr=0.001)  

In [20]:
model_scratch

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=500, bias=True)
  (fc3): Linear(in_features=500, out_features=50, bias=True)
  (dropout): Dropout(p=0.2)
)

### (IMPLEMENTATION) Model Architecture

Use transfer learning to create a CNN to classify images of landmarks.  Use the code cell below, and save your initialized model as the variable `model_transfer`.

In [21]:
## TODO: Specify model architecture

model_transfer = model_scratch

fc1_transfer = nn.Linear(64 * 28 * 28, 600) ## redefine the last fc layers
fc2_transfer = nn.Linear(600, 200)
fc3_transfer = nn.Linear(200, 50)

model_transfer.fc1=fc1_transfer ## overwrite
model_transfer.fc2=fc2_transfer
model_transfer.fc3=fc3_transfer

print(model_transfer)


#-#-# Do NOT modify the code below this line. #-#-#

if use_cuda:
    model_transfer = model_transfer.cuda()

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=600, bias=True)
  (fc2): Linear(in_features=600, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=50, bias=True)
  (dropout): Dropout(p=0.2)
)


__Question 3:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  Describe why you think the architecture is suitable for the current problem.

__Answer:__  

### (IMPLEMENTATION) Train and Validate the Model

Train and validate your model in the code cell below.  [Save the final model parameters](http://pytorch.org/docs/master/notes/serialization.html) at filepath `'model_transfer.pt'`.

In [14]:
# TODO: train the model and save the best model parameters at filepath 'model_transfer.pt'
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        print ("---n_epochs---")
        print(epoch)
        
        ###################
        # train the model #
        ###################
        # set the module to training mode
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            
            ##print (batch_idx)

            ## TODO: find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - train_loss))
            
            ####### clear the gradients of all optimized variables
            criterion.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            # update training loss
            ###### train_loss += loss.item()*data.size(0)
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - train_loss))           

        ######################    
        # validate the model #
        ######################
        # set the model to evaluation mode
        
        ######## Enumerate() method adds a counter to an iterable and returns it in
        ######## a form of enumerating object. 
        ######## This enumerated object can then be used directly for loops or converted 
        ######## into a list of tuples using the list() method.
        
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            ## TODO: update average validation loss 
            output = model_transfer(data) ###### for each image in validation set, calculate the prediction
            loss = criterion(output, target) ##### calculate the loss between output and target
            # update average validation loss 
            ## valid_loss += loss.item()*data.size(0)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - valid_loss))    
    
        # calculate average losses
        train_loss = train_loss/len(train_loader.sampler)
        valid_loss = valid_loss/len(valid_loader.sampler)            

        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
                epoch, 
                train_loss,
                valid_loss
             ))

        ## TODO: if the validation loss has decreased, save the model at the filepath stored in save_path
        
        # save model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), 'model_transfer.pt')
            valid_loss_min = valid_loss       
        
    return model

num_epochs=20
model_transfer.apply(custom_weight_init)
# train the model
model_transfer = train(num_epochs, loaders_transfer, model_transfer, get_optimizer_transfer(model_transfer),criterion_transfer, use_cuda, 'model_transfer.pt')

#-#-# Do NOT modify the code below this line. #-#-#

# load the model that got the best validation accuracy
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

NameError: name 'model_transfer' is not defined

### (IMPLEMENTATION) Test the Model

Try out your model on the test dataset of landmark images. Use the code cell below to calculate and print the test loss and accuracy.  Ensure that your test accuracy is greater than 60%.

In [13]:
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)

NameError: name 'loaders_transfer' is not defined

---
<a id='step3'></a>
## Step 3: Write Your Landmark Prediction Algorithm

Great job creating your CNN models! Now that you have put in all the hard work of creating accurate classifiers, let's define some functions to make it easy for others to use your classifiers.

### (IMPLEMENTATION) Write Your Algorithm, Part 1

Implement the function `predict_landmarks`, which accepts a file path to an image and an integer k, and then predicts the **top k most likely landmarks**. You are **required** to use your transfer learned CNN from Step 2 to predict the landmarks.

An example of the expected behavior of `predict_landmarks`:
```
>>> predicted_landmarks = predict_landmarks('example_image.jpg', 3)
>>> print(predicted_landmarks)
['Golden Gate Bridge', 'Brooklyn Bridge', 'Sydney Harbour Bridge']
```

In [28]:
from PIL import Image
import torchvision.transforms.functional as TF

image = Image.open('directory to extract/landmark_images/test/09.Golden_Gate_Bridge/190f3bae17c32c37.jpg')

img = data_transform(image)
img
type(img)
print(img.shape)
##crop = TF.center_crop(image, output_size=[224])
img.reshape(1,3,224,224)
print(img.shape)
##print (x)




torch.Size([3, 224, 224])
torch.Size([3, 224, 224])


In [29]:
model_transfer

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=600, bias=True)
  (fc2): Linear(in_features=600, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=50, bias=True)
  (dropout): Dropout(p=0.25)
)

In [33]:
import cv2
from PIL import Image
import torchvision.transforms.functional as TF

## the class names can be accessed at the `classes` attribute
## of your dataset object (e.g., `train_dataset.classes`)

def predict_landmarks(img_path, k):
    ## TODO: return the names of the top k landmarks predicted by the transfer learned CNN
    data_transform = transforms.Compose([transforms.RandomResizedCrop(224), transforms.ToTensor()])
    image=Image.open(img_path)
    data = data_transform(image)
    data.unsqueeze_(0)
    ##print(data.shape)
    pred=model_transfer(data)
    value, indexes= pred.topk(k)
    indexes_np=indexes.numpy().flatten() ##convert tensor to 1D array (flatten is to convert from 1D to 2D)
    names=[]
    for index in indexes_np:
        names.append(classes[index][3:])
    return names

# test on a sample image
predict_landmarks('directory to extract/landmark_images/test/09.Golden_Gate_Bridge/190f3bae17c32c37.jpg', 3)

RuntimeError: Expected object of type torch.FloatTensor but found type torch.cuda.FloatTensor for argument #2 'weight'

### (IMPLEMENTATION) Write Your Algorithm, Part 2

In the code cell below, implement the function `suggest_locations`, which accepts a file path to an image as input, and then displays the image and the **top 3 most likely landmarks** as predicted by `predict_landmarks`.

Some sample output for `suggest_locations` is provided below, but feel free to design your own user experience!
![](images/sample_landmark_output.png)

In [ ]:
def suggest_locations(img_path):
    # get landmark predictions
    predicted_landmarks = predict_landmarks(img_path, 3)  
    image=Image.open(img_path)
    ## TODO: display image and display landmark predictions
    predicted_landmarks[len(predicted_landmarks)-1]='or '+predicted_landmarks[len(predicted_landmarks)-1]
    text1="Is this picture of the" 
    text2=', '.join(predicted_landmarks)
    text=text1+"\n"+text2+'?'
    plt.imshow(image)
    plt.figtext(0.5, -0.05, text, wrap=False, horizontalalignment='center', fontsize=12)

    
# test on a sample image
suggest_locations('images/test/09.Golden_Gate_Bridge/190f3bae17c32c37.jpg')

### (IMPLEMENTATION) Test Your Algorithm

Test your algorithm by running the `suggest_locations` function on at least four images on your computer. Feel free to use any images you like.

__Question 4:__ Is the output better than you expected :) ?  Or worse :( ?  Provide at least three possible points of improvement for your algorithm.

__Answer:__ (Three possible points for improvement)

In [ ]:
## TODO: Execute the `suggest_locations` function on
## at least 4 images on your computer.
## Feel free to use as many code cells as needed.


suggest_locations('images/test/09.Golden_Gate_Bridge/190f3bae17c32c37.jpg')